In [36]:
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score 

In [15]:
df=pd.read_csv('Goal_Kicking_NRL 2020.csv')

#Add to dataframe
#Assumption 1: left foot kick = right foot kick

#adds negative left side component to kick
def x_cord(row):
    if row.Side == 'L':
        return -row.Width
    else:
        return row.Width

#numeric Result
def Result_Num(row):
    if row.Result == 'success':
        return 1
    else:
        return 0
    
#x_norm: mirror left foot kickers with right footers
def x_norm(row):
    if row.Foot == 'left':
        return -row.x_cord
    else:
        return row.x_cord
    

df['x_cord']=df.apply(lambda row: x_cord(row), axis=1)
df['Result_Num']=df.apply(lambda row: Result_Num(row), axis=1)
df['x_norm']=df.apply(lambda row: x_norm(row), axis=1)

print(df)


     Kick     MatchID  Rd  Cond    HomeTeam    AwayTeam  HomeSc  AwaySc  \
0       1  2020110110   1  Fine  Parramatta  Canterbury       0       0   
1       2  2020110110   1  Fine  Parramatta  Canterbury       0       2   
2       3  2020110110   1  Fine  Parramatta  Canterbury       0       2   
3       4  2020110110   1  Fine  Parramatta  Canterbury       6       2   
4       5  2020110120   1  Fine    Canberra  Gold Coast       4       0   
..    ...         ...  ..   ...         ...         ...     ...     ...   
123   124  2020110280   2  Fine  Gold Coast  Parramatta       6      24   
124   125  2020110280   2  Fine  Gold Coast  Parramatta       6      30   
125   126  2020110280   2  Fine  Gold Coast  Parramatta       6      36   
126   127  2020110280   2  Fine  Gold Coast  Parramatta       6      42   
127   128  2020110280   2  Fine  Gold Coast  Parramatta       6      44   

              Player        Team  ... Type Width  Side Dist   Result  \
0    Brandon Wakeham  Cante

In [33]:
goal_width=5.5

def x1_func(row):
    x_1 = row.x_norm - goal_width/2
    return x_1

def x2_func(row):
    x_2 = row.x_norm + goal_width/2
    return x_2

def h1_func(row):
    h_1 = math.sqrt(row.x_1**2 + row.Dist**2)
    return h_1

def h2_func(row):
    h_2 = math.sqrt(row.x_2**2 + row.Dist**2)
    return h_2

def angle_func(row):
    Angle = math.acos( (row.h_1**2 + row.h_2**2 - goal_width**2)/(2*row.h_1*row.h_2) )
    alpha = Angle * 180/math.pi
    return alpha

def distance(row):
    return math.sqrt(row.x_norm**2 + row.Dist**2)

df['x_1']=df.apply(lambda row: x1_func(row), axis=1)
df['x_2']=df.apply(lambda row: x2_func(row), axis=1)
df['h_1']=df.apply(lambda row: h1_func(row), axis=1)
df['h_2']=df.apply(lambda row: h2_func(row), axis=1)
df['alpha']=df.apply(lambda row: angle_func(row), axis=1)
df['distance']=df.apply(lambda row: distance(row), axis=1)

print(df)

     Kick     MatchID  Rd  Cond    HomeTeam    AwayTeam  HomeSc  AwaySc  \
0       1  2020110110   1  Fine  Parramatta  Canterbury       0       0   
1       2  2020110110   1  Fine  Parramatta  Canterbury       0       2   
2       3  2020110110   1  Fine  Parramatta  Canterbury       0       2   
3       4  2020110110   1  Fine  Parramatta  Canterbury       6       2   
4       5  2020110120   1  Fine    Canberra  Gold Coast       4       0   
..    ...         ...  ..   ...         ...         ...     ...     ...   
123   124  2020110280   2  Fine  Gold Coast  Parramatta       6      24   
124   125  2020110280   2  Fine  Gold Coast  Parramatta       6      30   
125   126  2020110280   2  Fine  Gold Coast  Parramatta       6      36   
126   127  2020110280   2  Fine  Gold Coast  Parramatta       6      42   
127   128  2020110280   2  Fine  Gold Coast  Parramatta       6      44   

              Player        Team  ...  Amount x_cord  Result_Num x_norm  \
0    Brandon Wakeham  Ca

In [39]:
#logistic regression
feature_cols=['alpha', 'distance']
X=df[feature_cols]
y=df['Result_Num']

model = LogisticRegression()

model.fit(X,y)


C:\Users\Stephen Sung\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)